<a href="https://colab.research.google.com/github/Mark-THU/load_forecast/blob/main/FNN_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip3 install xgboost --upgrade
# Using XGBoost for feature selection
# Using FNN for prediction of 24 hours
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdb
import torch
import torch.nn as nn

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, KFold
from sqlalchemy import create_engine
from xgboost import XGBRegressor
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader

Requirement already up-to-date: xgboost in /usr/local/lib/python3.7/dist-packages (1.3.3)


In [2]:
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"
device = torch.device(dev)

In [3]:
# load data
url = 'https://raw.githubusercontent.com/Mark-THU/load_forecast/main/integrate_0101510000.csv'
data = pd.read_csv(url, sep='\t', index_col='time')
data = data[['tembody','tem', 'dayType', 'weekType', 'load']]

In [4]:
# MinMax normalization
scaler = MinMaxScaler()
load = data[["load"]].values
scaler.fit(load)
scaler_all = MinMaxScaler()
data = scaler_all.fit_transform(data)

In [5]:
# build supervised data
def Series_To_Supervise(data, seq_len, y_col_index):
    """
    convert series data to supervised data
    :param data: original data
    :param seq_len: length of sequence
    :y_col_index: index of column which acts as output
    :return: return two ndarrays-- input and output in format suitable to feed to LSTM
    """
    dim_0 = data.shape[0] - seq_len
    dim_1 = data.shape[1]
    x = np.zeros((dim_0, seq_len, dim_1))
    y = np.zeros((dim_0,))
    for i in range(dim_0):
        x[i] = data[i: i+seq_len]
        y[i] = data[i+seq_len, y_col_index]
    print("shape of x: {}, shape of y: {}".format(x.shape, y.shape))
    return x, y

In [6]:
X, y = Series_To_Supervise(data, 48, 4)
X = X.reshape(X.shape[0], -1)

shape of x: (22769, 48, 5), shape of y: (22769,)


In [7]:
# define a XGBoost model which is trained for feature selection
model_xgb = XGBRegressor()
model_xgb.fit(X, y)
# select the most important 20 features
ind = (-model_xgb.feature_importances_).argsort()[0:20]

In [8]:
x, _ = Series_To_Supervise(data, 72, 4)
y = y[0:-24]
# split dataset into train, valid, test dataset
# train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.3, random_state=1)
# valid_x, test_x, valid_y, test_y = train_test_split(test_x, test_y, test_size=0.5, random_state=1)
# 5-fold cross-validation
def split_dataset(X, Y, n_split=5):
    """
    X: original feature, size * 72 * features
    Y: labels, size * 1
    return: list of train_x, test_x, train_y, test_y
    """
    kf = KFold(n_splits=n_split, shuffle=True, random_state=1)
    train_x_list = list()
    valid_x_list = list()
    test_x_list = list()
    train_y_list = list()
    valid_y_list = list()
    test_y_list = list()
    for train_index, test_index in kf.split(X):
        train_x_list.append(X[train_index])
        train_y_list.append(Y[train_index])
        test_x = X[test_index]
        test_y = Y[test_index]
        valid_x, test_x, valid_y, test_y = train_test_split(test_x, test_y, test_size=0.5, random_state=1)
        valid_x_list.append(valid_x)
        valid_y_list.append(valid_y)
        test_x_list.append(test_x)
        test_y_list.append(test_y)
    return train_x_list, valid_x_list, test_x_list, train_y_list, valid_y_list, test_y_list
train_x, valid_x, test_x, train_y, valid_y, test_y = split_dataset(x, y)

shape of x: (22745, 72, 5), shape of y: (22745,)


In [9]:
# define FNN model 
class FNN(nn.Module):
    """
    A fnn neural network
    """
    def __init__(self, hidden_dim):
        super(FNN, self).__init__()
        
        self.fnn = nn.Sequential(
            nn.Linear(20, 10),
            nn.ReLU(),
            nn.Linear(10, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim,1),
            nn.ReLU(),
        )
    
    def forward(self, x):
        out = self.fnn(x)
        out = out[:, -1]
        return out

In [10]:
# train the model 
def Train_Model(train_x, train_y, valid_x, valid_y, batch_size, lr, number_epoch, hidden_dim):
    model_fnn = FNN(hidden_dim)
    model_fnn.to(device=device)
    # choose the most important features
    train_x = train_x[:,0:48].reshape(train_x.shape[0], -1)[:, ind]
    valid_x = valid_x[:,0:48].reshape(valid_x.shape[0], -1)[:, ind]
    train_dataset = TensorDataset(torch.FloatTensor(train_x), torch.FloatTensor(train_y))
    valid_dataset = TensorDataset(torch.FloatTensor(valid_x), torch.FloatTensor(valid_y))
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model_fnn.parameters(), lr=lr)
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, drop_last=False)
    valid_loader = DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=True, drop_last=False)
    valid_loss_min = np.Inf
    num_without_imp = 0
    for epoch in range(1, number_epoch+1):
        for i, (inputs, labels) in enumerate(train_loader):  
            inputs = inputs.to(device=device)
            labels = labels.to(device=device)          
            optimizer.zero_grad()
            outputs = model_fnn(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            if i % 10 == 0:
                if num_without_imp > 20:
                  return model_fnn
                num_without_imp = num_without_imp + 1
                valid_losses = list()
                model_fnn.eval()
                for inp, lab in valid_loader:
                    inp = inp.to(device=device)
                    lab = lab.to(device=device)
                    out = model_fnn(inp)
                    valid_loss = criterion(out, lab)
                    valid_losses.append(valid_loss.item())
                
                model_fnn.train()
                print("Epoch: {}/{}...".format(epoch, number_epoch),
                     "Step: {}/{}...".format(i+1, len(train_loader)//batch_size),
                     "Loss: {:.8f}...".format(loss.item()),
                     "Valid Loss: {:.8f}...".format(np.mean(valid_losses)))
                if np.mean(valid_losses) < valid_loss_min:
                    num_without_imp = 0
                    torch.save(model_fnn.state_dict(), "fnn_state_dict.pt")
                    print("Valid loss decreased ({:.6f}-->{:.6f}). Saving model...".format(valid_loss_min, np.mean(valid_losses)))
                    valid_loss_min = np.mean(valid_losses)
    return model_fnn

In [11]:
# test model by hour
def Test_Model(model, test_x, test_y, batch_size, scaler):
    # choose the most important features
    test_x = test_x.reshape(test_x.shape[0], -1)[:, ind]
    test_dataset = TensorDataset(torch.FloatTensor(test_x), torch.FloatTensor(test_y))
    test_loader = DataLoader(dataset = test_dataset, batch_size=batch_size, shuffle=False, drop_last=False)
    model.load_state_dict(torch.load('fnn_state_dict.pt'))
    y_pred = list()
    y_true = list()
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device=device)
            labels = labels.to(device=device)
            outputs = model(inputs)
            y_pred = y_pred + outputs.cpu().numpy().tolist()
            y_true = y_true + labels.cpu().numpy().tolist()
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    y_pred = y_pred.reshape(-1, 1)
    y_true = y_true.reshape(-1, 1)
    load_pred = scaler.inverse_transform(y_pred)
    load_true = scaler.inverse_transform(y_true)
    MAPE = np.mean(np.abs(load_true-load_pred)/load_true)
    return MAPE

In [12]:
# test model by day
def Test_Model_24(model, test_x, test_y, scaler):
    test_dataset = TensorDataset(torch.FloatTensor(test_x), torch.FloatTensor(test_y))
    model.load_state_dict(torch.load('fnn_state_dict.pt'))
    size = 24
    y_pred = list()
    y_true = list()
    with torch.no_grad():
        for inputs, _ in test_dataset:
            inputs = inputs.to(device=device)
            outputs = list()
            labels = list()
            for i in range(size):
                inp = inputs[i:i+48,:]
                inp = inp.reshape(1, -1)[:, ind]
                out = model(inp)
                outputs.append(out.cpu().numpy().tolist())
                labels.append(inputs[i+48, 4].cpu().numpy().tolist())
                inputs[i+48, 4] = out
            y_pred.append(outputs)
            y_true.append(labels)
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    y_pred = y_pred.reshape(-1, 1)
    y_true = y_true.reshape(-1, 1)    
    load_pred = scaler.inverse_transform(y_pred)
    load_true = scaler.inverse_transform(y_true)
    MAPE = np.mean(np.abs(load_true-load_pred)/load_true)
    return MAPE

In [13]:
# model configs
def Model_Configs():
    batch_sizes = [512]
    lrs = [0.01]
    number_epochs = [50]
    hidden_dims = [5]
    configs = list()
    for i in batch_sizes:
        for j in lrs:
            for k in number_epochs:
                for l in hidden_dims:
                    configs.append([i, j ,k ,l])
    return configs

In [14]:
def main():
    print("model configs set")
    configs = Model_Configs()
    num_test = 5
    MAPE_list = list()
    MAPE_24_list = list()
    for config in configs:
        batch_size = config[0]
        lr = config[1]
        number_epoch = config[2]
        hidden_dim = config[3]
        print("Config: batch_size--{}, lr--{}, number_epoch--{}, hidden_dim--{}".format(config[0], config[1], config[2], config[3]))
        i = 0
        MAPE_sum = 0
        MAPE_sum_24 = 0
        while(i < num_test):
          model = Train_Model(train_x[i], train_y[i], valid_x[i], valid_y[i], batch_size, lr, number_epoch, hidden_dim)
          MAPE = Test_Model(model, test_x[i], test_y[i], batch_size, scaler=scaler)
          MAPE_24 = Test_Model_24(model, test_x[i].copy(), test_y[i], scaler)
          if MAPE < 0.2:
            i += 1
            MAPE_sum += MAPE
            MAPE_sum_24 += MAPE_24
        print("Test MAPE: {:.6f}".format(MAPE_sum/num_test),
             "Test MAPE 24: {:.6f}".format(MAPE_sum_24/num_test))
        MAPE_list.append(MAPE_sum/num_test)
        MAPE_24_list.append(MAPE_sum_24/num_test)
    return (MAPE_list, MAPE_24_list)

In [15]:
MAPE_list, MAPE_list_24 = main()

model configs set
Config: batch_size--512, lr--0.01, number_epoch--50, hidden_dim--5
Epoch: 1/50... Step: 1/0... Loss: 0.12622344... Valid Loss: 0.12832657...
Valid loss decreased (inf-->0.128327). Saving model...
Epoch: 1/50... Step: 11/0... Loss: 0.12640974... Valid Loss: 0.12891678...
Epoch: 1/50... Step: 21/0... Loss: 0.13478413... Valid Loss: 0.12923399...
Epoch: 1/50... Step: 31/0... Loss: 0.13600487... Valid Loss: 0.12854067...
Epoch: 2/50... Step: 1/0... Loss: 0.13306406... Valid Loss: 0.12838456...
Epoch: 2/50... Step: 11/0... Loss: 0.12978765... Valid Loss: 0.12805070...
Valid loss decreased (0.128327-->0.128051). Saving model...
Epoch: 2/50... Step: 21/0... Loss: 0.12911467... Valid Loss: 0.12862017...
Epoch: 2/50... Step: 31/0... Loss: 0.12921947... Valid Loss: 0.12793983...
Valid loss decreased (0.128051-->0.127940). Saving model...
Epoch: 3/50... Step: 1/0... Loss: 0.13630685... Valid Loss: 0.12856349...
Epoch: 3/50... Step: 11/0... Loss: 0.13114537... Valid Loss: 0.12745

In [16]:
configs = Model_Configs()
best_config = configs[np.argmin(MAPE_list)]
for i in range(len(configs)):
    print("config: {}, MAPE: {:.6f}".format(configs[i], MAPE_list[i]))
print("best config: {}, MAPE: {:.6f}".format(best_config, min(MAPE_list)))
best_config = configs[np.argmin(MAPE_list_24)]
for i in range(len(configs)):
    print("config: {}, MAPE_24: {:.6f}".format(configs[i], MAPE_list_24[i]))
print("best config: {}, MAPE_24: {:.6f}".format(best_config, min(MAPE_list_24)))

config: [512, 0.01, 50, 5], MAPE: 0.011723
best config: [512, 0.01, 50, 5], MAPE: 0.011723
config: [512, 0.01, 50, 5], MAPE_24: 0.025421
best config: [512, 0.01, 50, 5], MAPE_24: 0.025421
